<a href="https://colab.research.google.com/github/anupa35/Machine-Learning/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 2



# 1. Preprocessing

In [20]:
import pandas as pd
import numpy as np

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
df = pd.read_csv("/content/drive/MyDrive/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [23]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [24]:
df.info() #checking data types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [25]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce') # changing incorrect data type of TotalCharges
df.isnull().sum() # checking for null values

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [26]:
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median()) # Handling missing values of TotalCharges

In [27]:
df.isnull().sum() # checking for null values again

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [28]:
df = df.drop(columns=['customerID']) # removing the customerID column since it is not useful

In [29]:
# converting Yes/No containing columns into 1/0 values
columnswith_YesNo = ['Partner', 'Dependents', 'PhoneService','PaperlessBilling', 'Churn']

for col in columnswith_YesNo:
    df[col] = df[col].map({'Yes':1, 'No':0})

In [30]:
# converting gender column data into 1/0
df['gender'] = df['gender'].map({'Male':1, 'Female':0})

In [31]:
# target
X = df.drop('Churn', axis=1)
y = df['Churn']

In [32]:
# splitting into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [41]:
columns_multicatego = ['InternetService', 'Contract', 'PaymentMethod', 'MultipleLines','OnlineSecurity', 'OnlineBackup',
                       'DeviceProtection', 'TechSupport','StreamingTV', 'StreamingMovies']

In [42]:
columnswith_numeric = ['tenure', 'MonthlyCharges', 'TotalCharges']

In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [43]:
# For neural network
# using OneHotEncoding for columns containing multiple categories and scaling numeric columns

preprocessor_nn = ColumnTransformer([
    ('categorical', OneHotEncoder(handle_unknown='ignore'), columns_multicatego),
    ('numeric', StandardScaler(), columnswith_numeric)
])

In [44]:
# For Decision Tree
# using OneHotEncoding for columns containing multiple categories, no scaling
preprocessor_tree = ColumnTransformer([
    ('categorical', OneHotEncoder(handle_unknown='ignore'), columns_multicatego),
    ('numeric', 'passthrough', columnswith_numeric)  # no scaling
])

In [46]:
# For Neural network
# Fitting and transforming data

X_train_nn = preprocessor_nn.fit_transform(X_train)
X_test_nn = preprocessor_nn.transform(X_test)


In [47]:
# For Decision Tree
# Fitting and transforming data

X_train_tree = preprocessor_tree.fit_transform(X_train)
X_test_tree = preprocessor_tree.transform(X_test)

# Feature Engineering